Playing with web3.py

Install Depedencies

In [7]:
import etherscan
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt # importing matplotlib
import numpy as np # importing numpy
import schedule
import time
import json
import requests

from dotenv import load_dotenv
from datetime import datetime
from web3 import Web3
from os import getenv
import os
from tqdm import tqdm
from decimal import Decimal

Connecting to web3 using Infura

In [9]:
load_dotenv()


es = etherscan.Client(api_key=getenv('ETHERSCAN'))
w3 = Web3(Web3.HTTPProvider(getenv('INFURA')))

w3.isConnected()

True

In [10]:
# address book
carlisle = '0xd791427e372186E90637caF1D6D6AFb054BeeC7A'
rarible_contract = '0xcd4EC7b66fbc029C116BA9Ffb3e59351c20B5B06'
opensea = '0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b'
foundation = '0xcDA72070E455bb31C7690a170224Ce43623d0B6f'
superrare = '0x41A322b28D0fF354040e2CbC676F0320d8c8850d'
knownorigin = '0x848b0EA643e5a352D78e2c0c12A2dD8c96feC639'

In [11]:
def get_balance(wallet):
    wei = w3.eth.getBalance(wallet)
    return w3.fromWei(wei, 'ether')

get_balance(carlisle)

Decimal('0.050000416012272068')

In [ ]:
block = w3.eth.getBlock('latest')
block

In [17]:
transaction = '0x7a04337a420fede66c1038c3288b250c8e1ffd3fd6de46a3ec44a09150d803b8'

contract = w3.eth.getTransaction(transaction)
contract

AttributeDict({'blockHash': HexBytes('0xc190a41f539d01561b987161ee34f4cbf77112e3f5631974abb97db6bbdedd53'),
 'blockNumber': 12292666,
 'from': '0x81085713E6dB7b9511b85CE5ed33C790980BFA36',
 'gas': 203534,
 'gasPrice': 140000000000,
 'hash': HexBytes('0x7a04337a420fede66c1038c3288b250c8e1ffd3fd6de46a3ec44a09150d803b8'),
 'input': '0xa0712d6800000000000000000000000000000000000000000000000dcc988cf91e06d800',
 'nonce': 9,
 'r': HexBytes('0x2440d9482e3b6a7225300f2361057bb6b59c7cdfcb334546ac1e16738e5cb0f9'),
 's': HexBytes('0x23ce84bbdcc4f8fa1102bfaedad9c616a72c22626c7608b6b64b1c3b101634bd'),
 'to': '0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643',
 'transactionIndex': 312,
 'type': '0x0',
 'v': 38,
 'value': 0})

In [8]:
rarible = pd.read_csv('rarible.csv', index_col=0)

In [9]:
def mp_transactions(address):
    columns = ['timestamp', 'block_number', 'from', 'to', 'input', 'hash', 'value', 'gas', 
                'gas_price', 'gas_used', 'nonce', 'confirmations', 'is_error', 'tx_receipt_status', 
                'transaction_index', 'cumulative_gas_used', 'block_hash']

    transactions = []

    block = w3.eth.getBlock('latest')

    if rarible['block_number'].empty:
        start_block = block.number - 100
    else:
        start_block = rarible['block_number'].iloc[-1]

    for l in tqdm(es.get_transactions_by_address(address, start_block=start_block, limit=9999)):
        time.sleep(.2)
        transactions.append(l)

    df = pd.DataFrame(transactions, columns=columns)
    df['timestamp'] = df['timestamp'].apply(datetime.fromtimestamp)
    df = df[df['value'] != 0]
    df = df[df['is_error'] != True]
    df['value'] = [w3.fromWei(x, 'ether') for x in df['value']]
    df = df.reset_index(drop=True)
    if not os.path.isfile('rarible.csv'):
        df.to_csv('rarible.csv', header='column_names', index=False)
    else:
        df.to_csv('rarible.csv', mode='a', header=False, index=False)
    return df

In [10]:
mp_transactions(rarible_contract)

100%|██████████| 23/23 [00:04<00:00,  4.87it/s]


,timestamp,block_number,from,to,input,hash,value,gas,gas_price,gas_used,nonce,confirmations,is_error,tx_receipt_status,transaction_index,cumulative_gas_used,block_hash
0,2021-04-24 13:53:59,12304673,0xfee85cc9d00f373a159fab2215308c2ae4ae1428,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000000f75caa45db7...,0xa0296b5cea402ea3db668fadc9e95f422a456b32b15e...,0.046125,249739,53000000000,249739,15,95,False,True,85,12563333,0xa893b6370180e9e857c213a858d93a85fc5a5e5f3ea7...
1,2021-04-24 13:54:10,12304675,0xd6395c768b5008ef8779b6525d7d903603e15b5c,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000b1fa25350cd0...,0xca673379719b005f1229c68f2506fa36924142276927...,0.00189625,249751,53000000000,249751,3,93,False,True,73,5769993,0xc0a2f385eeb37803a691948dcb3d2897165699d144d1...
2,2021-04-24 13:54:10,12304675,0x36ed2d75a82e180e0871456b15c239b73b4ee9f4,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec639200000000000000000000000086ec43fd8aa1...,0xc464448801c57aafbf5b422f8d01f2e51846f380901a...,0.005125,266839,53000000000,266839,2519,93,False,True,103,8921681,0xc0a2f385eeb37803a691948dcb3d2897165699d144d1...
3,2021-04-24 13:54:23,12304677,0xf35fdc03030ac54840c4c2c4ac38dd873795cec2,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000008d003a32c096...,0xf3d3804a5e4bb8e2d675d52ec86d8d8ed92609df52b7...,0.01025,249715,52000000000,249715,207,91,False,True,113,9614585,0x0c0f0434e20553a1a3faf06c0852408de79a9381fa45...
4,2021-04-24 13:58:26,12304695,0xe9d2dca3ce579e181255edd661f95a6fcaccd31d,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000d8e0ba725055...,0xc8f0e58b39bbfb2ed11b90902def83abe7243025f9f0...,0.1025,266827,61000000000,266827,0,73,False,True,207,13415321,0xb72b4836c4032d2d3e20c758f1adcbd411055410ede4...
5,2021-04-24 14:00:07,12304703,0x4d7d960252eedd37b7d73c6915da37ba8c99bb69,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000008431f7c99d42...,0x7ef5957ed4ee5105a75b3a3ef178f026ccc268d8dcfa...,0.03075,249715,52000000000,249715,21,65,False,True,107,11081765,0xb132bef0d9ed4dfd75a57c8cc092369e4615f42f3df8...
6,2021-04-24 14:00:55,12304709,0x194618ff8547beb766d60071ef200f71df1f47e2,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000113f2cf739d7...,0xa23fce4a5ae712143141c3aaefc6dae9f9f9cb4b8c04...,0.01025,249739,52000000000,249739,4,59,False,True,117,11331699,0xa2dfea8e95198d14eb715c2b3ae387931e440eefbb3f...
7,2021-04-24 14:01:36,12304713,0x724b1c8a6bf6bf0681b3f8c312a3e61a9364de06,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000979f8cdc881f...,0xfc627f82f33792e94da77afb211be5de800db227bc8c...,0.15375,319474,52000000000,274474,20,55,False,True,43,9659843,0x05a8239b7c05a462af67f958cb9583b2522c53749b07...
8,2021-04-24 14:02:00,12304716,0x9854990358e62ff3f0a62d14e2e1df5e729c95f9,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000a12235589204...,0xa2ed0eee0f95aba32dadd43d30d3840e20d752c0c516...,0.15375,266803,120000000000,266803,184,52,False,True,59,3460950,0xfa2297452a079fd437a1baa5877dd3d29832535619cc...
9,2021-04-24 14:03:50,12304720,0xe6f3f846390f36efc840b9a3e6148c340d9d39e6,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000abe67ca4eefb...,0xe3df32e8918b27e625622d46c626d9b26ac12ea0c0a3...,0.5125,249739,67000000000,234739,371,48,False,True,264,11075445,0x58263cd86d61fa9b177852fd831ebc574316246c276f...


In [11]:
rarible_df = pd.read_csv('rarible.csv')
rarible_df

,timestamp,block_number,from,to,input,hash,value,gas,gas_price,gas_used,...,249739,53000000000,249739.1,15,95,False,True,85,12563333,0xa893b6370180e9e857c213a858d93a85fc5a5e5f3ea72d70230d214f53d0baa9
0,2021-04-24 13:54:10,12304675,0xd6395c768b5008ef8779b6525d7d903603e15b5c,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000b1fa25350cd0...,0xca673379719b005f1229c68f2506fa36924142276927...,0.001896,249751,53000000000,249751,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-24 13:54:10,12304675,0x36ed2d75a82e180e0871456b15c239b73b4ee9f4,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec639200000000000000000000000086ec43fd8aa1...,0xc464448801c57aafbf5b422f8d01f2e51846f380901a...,0.005125,266839,53000000000,266839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-24 13:54:23,12304677,0xf35fdc03030ac54840c4c2c4ac38dd873795cec2,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000008d003a32c096...,0xf3d3804a5e4bb8e2d675d52ec86d8d8ed92609df52b7...,0.010250,249715,52000000000,249715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-24 13:58:26,12304695,0xe9d2dca3ce579e181255edd661f95a6fcaccd31d,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000d8e0ba725055...,0xc8f0e58b39bbfb2ed11b90902def83abe7243025f9f0...,0.102500,266827,61000000000,266827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-24 14:00:07,12304703,0x4d7d960252eedd37b7d73c6915da37ba8c99bb69,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec63920000000000000000000000008431f7c99d42...,0x7ef5957ed4ee5105a75b3a3ef178f026ccc268d8dcfa...,0.030750,249715,52000000000,249715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-04-24 14:00:55,12304709,0x194618ff8547beb766d60071ef200f71df1f47e2,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000113f2cf739d7...,0xa23fce4a5ae712143141c3aaefc6dae9f9f9cb4b8c04...,0.010250,249739,52000000000,249739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-04-24 14:01:36,12304713,0x724b1c8a6bf6bf0681b3f8c312a3e61a9364de06,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000979f8cdc881f...,0xfc627f82f33792e94da77afb211be5de800db227bc8c...,0.153750,319474,52000000000,274474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-04-24 14:02:00,12304716,0x9854990358e62ff3f0a62d14e2e1df5e729c95f9,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000a12235589204...,0xa2ed0eee0f95aba32dadd43d30d3840e20d752c0c516...,0.153750,266803,120000000000,266803,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-04-24 14:03:50,12304720,0xe6f3f846390f36efc840b9a3e6148c340d9d39e6,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000abe67ca4eefb...,0xe3df32e8918b27e625622d46c626d9b26ac12ea0c0a3...,0.512500,249739,67000000000,234739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-04-24 14:05:18,12304727,0xa84ec76b0d783e68a844565f450c163097a182ea,0xcd4ec7b66fbc029c116ba9ffb3e59351c20b5b06,0x9cec6392000000000000000000000000a922df1e9044...,0x18428f6d1ae38bab968d6e811face9d8f8fe2138135a...,2.870000,266839,52000000000,266839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
fig = go.Figure()

value = rarible_df['value']
timestamp = rarible_df['timestamp']
a_hash = rarible_df['hash']
mean = rarible_df['value'].mean()

fig.add_trace(go.Scatter(y=value, x=timestamp, name=str(a_hash), mode='markers', text=rarible_df.hash,
                marker=dict(size=value.astype(float), color=value, colorscale='Cividis', showscale=True, line=dict(width=.5,
                                        color='grey'))))

fig.add_shape(type='line', x0=min(timestamp), y0=30, x1=max(timestamp), y1=30,
                line=dict(color='Red') ,xref='x', yref='y')

fig.update_layout(yaxis_title="", xaxis_title="<i>contract: 0xcd4EC7b66fbc029C116BA9Ffb3e59351c20B5B06</i>", width=600, height=900, yaxis=dict(zeroline=False),
                    template='seaborn', legend_title="Comparison", paper_bgcolor='rgba(0,0,0,0)', 
                    font=dict(family="Tahoma", size=10), plot_bgcolor='rgba(0,0,0,0)',
                    title=("Rarible<br>" +
                    "<i>Value of transactions including bids and sales</i>"))

# Add images
fig.add_layout_image(dict(source="gradient.png", xref="paper", yref="paper",
                        x=-1.5, y=1.2, sizex=6, sizey=6, layer="below"))

#fig.add_annotation(x=rarible_df['timestamp'].iloc[2170], y=70.5,
#            text="", showarrow=False, arrowhead=1)

fig.update_shapes(dict(xref='x', yref='y'))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_traces(marker_showscale=False)
fig.show()

In [ ]:
def hist_for_count(data):
    plt.style.use("fast")
    plt.figure(figsize=(21,14), facecolor=('#fff9f4')) # Make it 14x7 inch

    x = data # normal distribution
    n, bins, patches = plt.hist(x, bins=80, facecolor='#2ab0ff', edgecolor='#e0e0e0', linewidth=0.5, alpha=0.7)
    n = n.astype('int') # it MUST be integer

    # Good old loop. Choose colormap of your taste
    for i in range(len(patches)):
        patches[i].set_facecolor(plt.cm.viridis(n[i]/max(n)))

    # Add title and labels with custom font sizes
    plt.title('', fontsize=12)
    plt.xlabel('', fontsize=8)
    plt.ylabel('', fontsize=8)
    plt.grid(b=None)
    plt.savefig('foo.png')
    plt.show();
    return

In [ ]:
hist_for_count(foundation_df['timestamp'])

In [4]:
# OMG Address
abi = json.loads('[{"constant":true,"inputs":[],"name":"mintingFinished","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[],"name":"unpause","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_amount","type":"uint256"}],"name":"mint","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"paused","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"}],"name":"balanceOf","outputs":[{"name":"balance","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[],"name":"finishMinting","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":false,"inputs":[],"name":"pause","outputs":[{"name":"","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transfer","outputs":[],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_amount","type":"uint256"},{"name":"_releaseTime","type":"uint256"}],"name":"mintTimelocked","outputs":[{"name":"","type":"address"}],"payable":false,"type":"function"},{"constant":true,"inputs":[{"name":"_owner","type":"address"},{"name":"_spender","type":"address"}],"name":"allowance","outputs":[{"name":"remaining","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"type":"function"},{"anonymous":false,"inputs":[{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Mint","type":"event"},{"anonymous":false,"inputs":[],"name":"MintFinished","type":"event"},{"anonymous":false,"inputs":[],"name":"Pause","type":"event"},{"anonymous":false,"inputs":[],"name":"Unpause","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"owner","type":"address"},{"indexed":true,"name":"spender","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"from","type":"address"},{"indexed":true,"name":"to","type":"address"},{"indexed":false,"name":"value","type":"uint256"}],"name":"Transfer","type":"event"}]')
# OMG ABI
address = '0xd26114cd6EE289AccF82350c8d8487fedB8A0C07'

contract = w3.eth.contract(address=address, abi=abi)

totalSupply = contract.functions.totalSupply().call()
print(w3.fromWei(totalSupply, 'ether'))
print(contract.functions.name().call())
print(contract.functions.symbol().call())
balance = contract.functions.balanceOf('0xd26114cd6EE289AccF82350c8d8487fedB8A0C07').call()
print(w3.fromWei(balance, 'ether'))

140245398.245132780789239631
OMGToken
OMG
26254.40668528280698004


In [ ]:
signed_tx = w3.eth.account.signTransaction(tx, private_key)

tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)

print(w3.toHex(tx_hash))